In [ ]:
# Import necessary libraries and path relative to project
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import IsolationForest

import sys
import os

sys.path.append(os.path.join(os.path.abspath(''), '../pyno/lib'))

from autoencoder import Autoencoder

In [ ]:
data_file = "https://happy-research.s3.ap-southeast-1.amazonaws.com/annthyroid.csv"
chunk_size = 1000
percent_anomaly = 0.05
percent_training = 0.7
data = pd.DataFrame()

for i, chunk in enumerate(pd.read_csv(data_file, header=None, chunksize=chunk_size)):
    data = data.append(chunk)

input_dim = len(data.columns) - 1

training_partition = data.sample(frac=percent_training)
validation_partition = data.drop(training_partition.index)

print(training_partition[training_partition[input_dim] == -1])

training_data = training_partition.iloc[:,:input_dim]
validation_data = data.drop(training_data.index)

validation_data_raw = validation_data.iloc[:,:input_dim]
validation_data_labels = validation_data.iloc[:,-1].to_numpy()

In [ ]:
random_states = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]

for random_state in random_states:
    clf = IsolationForest(random_state=random_state)
    clf.fit(training_data)
    predictions = clf.fit_predict(validation_data_raw)
    tn, fp, fn, tp = np.array(confusion_matrix(validation_data_labels, predictions).ravel(), dtype=np.float64)
    print(tn, fp, fn, tp)